<a href="https://colab.research.google.com/github/AgboolaMubarak/Youtube-Video-Critic-Agent/blob/main/Youtube_Video_Critic_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt-dlp transformers peft accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.7 MB/s eta 0:00:00


In [ ]:
!pip install smolagents -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
# STEP 2: Imports
import os
import torch
from typing import Literal
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from smolagents import tool
from smolagents import ToolCallingAgent

In [ ]:
# STEP 3: Define Tools as Functions

@tool
def get_transcript(url: str) -> str:
    """
    Extracts the transcript from a YouTube video using yt-dlp.

    Args:
        url (str): The URL of the YouTube video.

    Returns:
        str: The extracted transcript text or a fallback message.
    """
    os.system(f"yt-dlp --write-auto-sub --sub-lang en --skip-download -o 'video' {url}")
    try:
        with open("video.en.vtt", "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return "Transcript not available."


@tool
def summarize_transcript(transcript: str) -> str:
    """
    Summarizes a transcript into a short paragraph.

    Args:
        transcript (str): The full transcript text.

    Returns:
        str: A summarized version of the transcript.
    """
    model_id = "google/flan-t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    input_text = "summarize: " + transcript[:1000]
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


@tool
def analyze_sentiment(text: str) -> Literal["Positive", "Negative"]:
    """
    Performs sentiment analysis on a given summary.

    Args:
        text (str): The summary text to analyze.

    Returns:
        Literal["Positive", "Negative"]: The sentiment classification.
    """
    model_id = "distilbert-base-uncased-finetuned-sst-2-english"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    outputs = model(**inputs)
    label = torch.argmax(outputs.logits, dim=1).item()
    return ["Negative", "Positive"][label]


@tool
def generate_review(summary_and_sentiment: str) -> str:
    """
    Generates a critic-style review using a summary and sentiment input.

    Args:
        summary_and_sentiment (str): A combined summary and sentiment string.

    Returns:
        str: A review of the video in a critical tone.
    """
    model_id = "google/flan-t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    prompt = "Generate a critical review: " + summary_and_sentiment
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# STEP 4: Assemble Multi-Agent Using smolagents
from smolagents import HfApiModel

model = HfApiModel(model_id="HuggingFaceH4/zephyr-7b-beta")

# STEP 4: Assemble and run the agent
agent = ToolCallingAgent(
    tools=[get_transcript, summarize_transcript, analyze_sentiment, generate_review],
    model=model,
    name="YouTubeCriticAgent",
    description="Multi-agent critic that reviews YouTube videos using transcript analysis.",
    max_steps=5
)

In [ ]:
# STEP 5: Run the agent
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
response = agent.run(f"""
Step 1: get_transcript({video_url})
Step 2: summarize_transcript(<Step 1>)
Step 3: analyze_sentiment(<Step 2>)
Step 4: generate_review(Summary: <Step 2>. Sentiment: <Step 3>)
""")

print("\n Final Critic Review:\n")
print(response)

╭───────────────────────────────────────── New run - YouTubeCriticAgent ──────────────────────────────────────────╮
│                                                                                                                 │
│ Step 1: get_transcript(https://www.youtube.com/watch?v=dQw4w9WgXcQ)                                             │
│ Step 2: summarize_transcript(<Step 1>)                                                                          │
│ Step 3: analyze_sentiment(<Step 2>)                                                                             │
│ Step 4: generate_review(Summary: <Step 2>. Sentiment: <Step 3>)                                                 │
│                                                                                                                 │
╰─ HfApiModel - HuggingFaceH4/zephyr-7b-beta ─────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_transcript' with arguments: {'url': 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'}           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: WEBVTT
Kind: captions
Language: en

00:00:01.360 --> 00:00:03.040
|♪♪♪]

00:00:18.640 --> 00:00:21.880
♪ We're no strangers to love ♪

00:00:22.640 --> 00:00:26.960
♪ You know the rules
and so do I ♪

00:00:27.040 --> 00:00:31.040
♪ A full commitment's
what I'm thinking of ♪

00:00:31.120 --> 00:00:35.080
♪ You wouldn't get this
from any other guy ♪

00:00:35.160 --> 00:00:39.520
♪ I just wanna tell you
how I'm feeling ♪

00:00:40.520 --> 00:00:42.920
♪ Gotta make you understand ♪

00:00:43.000 --> 00:00:45.120
♪ Never gonna give you up ♪

00:00:45.200 --> 00:00:47.080
♪ Never gonna let you down ♪

00:00:47.320 --> 00:00:51.120
♪ Never gonna run around
and desert you ♪

00:00:51.480 --> 00:00:53.480
♪ Never gonna make you cry ♪

00:00:53.600 --> 00:00:55.520
♪ Never gonna say goodbye ♪

00:00:55.720 --> 00:00:59.360
♪ Never gonna tell a lie
and hurt you ♪

00:01:00.800 --> 00:01:04.800
♪ We've known each other
for so long ♪

00:01:04.880 --> 00:01:09.040
♪ Your heart's been aching
but you're too shy to say it ♪

00:01:09.120 --> 00:01:12.960
♪ Inside we both know
what's been going ♪

00:01:13.360 --> 00:01:17.200
♪ We know the game
and we're gonna play it ♪

00:01:17.400 --> 00:01:22.040
♪ And if you ask me
how I'm feeling ♪

00:01:22.400 --> 00:01:25.240
♪ Don't tell me
you're too blind to see ♪

00:01:25.320 --> 00:01:27.280
♪ Never gonna give you up ♪

00:01:27.360 --> 00:01:29.320
♪ Never gonna let you down ♪

00:01:29.440 --> 00:01:33.720
♪ Never gonna run around
and desert you ♪

00:01:33.800 --> 00:01:35.600
♪ Never gonna make you cry ♪

00:01:35.760 --> 00:01:38.000
♪ Never gonna say goodbye ♪

00:01:38.080 --> 00:01:42.040
♪ Never gonna tell a lie
and hurt you ♪

00:01:42.200 --> 00:01:44.120
♪ Never gonna give you up ♪

00:01:44.280 --> 00:01:46.360
♪ Never gonna let you down ♪

00:01:46.480 --> 00:01:50.080
♪ Never gonna run around
and desert you ♪

00:01:50.760 --> 00:01:52.720
♪ Never gonna make you cry ♪

00:01:52.800 --> 00:01:54.680
♪ Never gonna say goodbye ♪

00:01:54.960 --> 00:01:58.760
♪ Never gonna tell a lie
and hurt you ♪

00:01:59.840 --> 00:02:02.960
♪ (Ooh, give you up) ♪

00:02:03.720 --> 00:02:07.360
♪ (Ooh, give you up) ♪

00:02:08.480 --> 00:02:10.120
♪ Never gonna give,
never gonna give ♪

00:02:10.240 --> 00:02:11.560
♪ (Give you up) ♪

00:02:12.480 --> 00:02:14.240
♪ Never gonna give,
never gonna give ♪

00:02:14.360 --> 00:02:15.920
♪ (Give you up) ♪

00:02:16.760 --> 00:02:21.080
♪ We've known each other
for so long ♪

00:02:21.200 --> 00:02:25.200
♪ Your heart's been aching
but you're too shy to say it ♪

00:02:25.280 --> 00:02:29.120
♪ Inside we both know
what's been going ♪

00:02:29.520 --> 00:02:33.200
♪ We know the game
and we're gonna play it ♪

00:02:33.360 --> 00:02:38.040
♪ I just wanna tell you
how I'm feeling ♪

00:02:38.640 --> 00:02:41.320
♪ Gotta make you understand ♪

00:02:41.400 --> 00:02:43.360
♪ Never gonna give you up ♪

00:02:43.440 --> 00:02:45.640
♪ Never gonna let you down ♪

00:02:45.720 --> 00:02:49.720
♪ Never gonna run around
and desert you ♪

00:02:49.800 --> 00:02:51.640
♪ Never gonna make you cry ♪

00:02:51.800 --> 00:02:53.960
♪ Never gonna say goodbye ♪

00:02:54.040 --> 00:02:57.600
♪ Never gonna tell a lie
and hurt you ♪

00:02:58.200 --> 00:03:00.240
♪ Never gonna give you up ♪

00:03:00.320 --> 00:03:02.440
♪ Never gonna let you down ♪

00:03:02.520 --> 00:03:06.640
♪ Never gonna run around
and desert you ♪

00:03:06.720 --> 00:03:08.720
♪ Never gonna make you cry ♪

00:03:08.840 --> 00:03:10.720
♪ Never gonna say goodbye ♪

00:03:10.840 --> 00:03:15.200
♪ Never gonna tell a lie
and hurt you ♪

00:03:15.280 --> 00:03:17.120
♪ Never gonna give you up ♪

00:03:17.200 --> 00:03:19.240
♪ Never gonna let you down ♪

00:03:19.400 --> 00:03:23.120
♪ Never gonna run around
and desert you ♪

00:03:23.360 --> 00:03:25.600
♪ Never gonna make you cry ♪

00:03:25.680 --> 00:03:27.840
♪ Never gonna say goodbye ♪

00:03:27.920 --> 00:03:31.320
♪ Never go

[Step 1: Duration 10.27 seconds| Input tokens: 1,758 | Output tokens: 48]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Based on the analysis of the summarized transcript     │
│ using sentiment analysis, this video is a heartfelt confession of everlasting love and devotion between two     │
│ individuals. The emotions expressed are unwavering and passionate, making it clear that this love is            │
│ unbreakable and true. The conclusion is indisputable - this video is a celebration of eternal love.'}           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Based on the analysis of the summarized transcript using sentiment analysis, this video is a 
heartfelt confession of everlasting love and devotion between two individuals. The emotions expressed are 
unwavering and passionate, making it clear that this love is unbreakable and true. The conclusion is indisputable -
this video is a celebration of eternal love.

[Step 4: Duration 7.54 seconds| Input tokens: 22,398 | Output tokens: 244]


 Final Critic Review:

Based on the analysis of the summarized transcript using sentiment analysis, this video is a heartfelt confession of everlasting love and devotion between two individuals. The emotions expressed are unwavering and passionate, making it clear that this love is unbreakable and true. The conclusion is indisputable - this video is a celebration of eternal love.


In [ ]:
from huggingface_hub import login
login("insert hugging face api")